In [1]:
import cv2
import numpy as np
import pandas as pd
import itertools

# Códigos detectores

In [2]:
def detectHarrisKeypoints(image, threshold=0.01, blockSize=2, ksize=3, k=0.04):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    gray_image_f32 = np.float32(gray_image)

    # Applying the function 
    dst = cv2.cornerHarris(gray_image_f32, blockSize, ksize, k) 
  
    # dilate to mark the corners 
    dst = cv2.dilate(dst, None)
    
    ret, dst = cv2.threshold(dst,threshold*dst.max(),255,0)
    dst = np.uint8(dst)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(gray_image_f32,np.float32(centroids),(5,5),(-1,-1),criteria)

    # # extract keypoints
    # points = np.argwhere(dst > threshold * dst.max())
    
    keypoints = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in corners]

    # draw keypoints
    # image[dst > threshold * dst.max()] = [0, 255, 0]
    kp_image = cv2.drawKeypoints(image, keypoints, None, color=(255, 0, 0), flags=0)

    return keypoints, kp_image


In [3]:
def detectSIFTKeypoints(image, nfeatures=0, nOctaveLayers=3, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6, enable_precise_upscale=False):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
    # Applying the function 
    sift = cv2.SIFT_create(nfeatures, nOctaveLayers, contrastThreshold, edgeThreshold, sigma, enable_precise_upscale) 
    kp, des = sift.detectAndCompute(gray_image, None) 
    
    # Applying the function 
    kp_image = cv2.drawKeypoints(image, kp, None, color=(0, 255, 0), flags=0) 

    return kp, kp_image 

In [4]:
def detectStarKeypoints(image, max_size = 41, response_threshold = 30, line_threshold_projected = 10,
                        line_threshold_binarized = 8, suppress_nonmax_size = 5):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
    # Applying the function 
    star = cv2.xfeatures2d.StarDetector_create(maxSize= max_size, 
                                        responseThreshold = response_threshold,
                                        lineThresholdProjected = line_threshold_projected,
                                        lineThresholdBinarized = line_threshold_binarized,
                                        suppressNonmaxSize = suppress_nonmax_size)
    kp = star.detect(gray_image, None)    

    # Applying the function 
    kp_image = cv2.drawKeypoints(image, kp, None, color=(0, 0, 255), flags=0) 

    return kp, kp_image 

In [5]:
# image = cv2.imread('../data/imgs/dsc07631.jpg')
# kp, kp_image = detectHarrisKeypoints(image, threshold=0.01, blockSize=2, ksize=3, k=0.02)
# print(len(kp))
# cv2.imshow('Star', kp_image) 
# cv2.waitKey() 

## Código comparando pontos

In [6]:
import numpy as np
from sklearn.metrics import pairwise_distances_argmin_min

def computeDistacesKeypoints(pts1, pts2, threshold=1):
    kps1 = [(kp.pt[0], kp.pt[1]) for kp in pts1]
    kps2 = [(kp.pt[0], kp.pt[1]) for kp in pts2]

    array_pts1 = np.asarray(kps1)
    array_pts2 = np.asarray(kps2)

    if array_pts1.shape[0] > 0 and array_pts2.shape[0] > 0:
        dists = pairwise_distances_argmin_min(array_pts1, array_pts2)  
        matches = [pts2[pt] for pt, dist in zip(dists[0], dists[1]) if dist <= threshold]
    else:
        matches = []
        
    return matches

## Primeiro experimento
- Rodando com os parâmetros default dos algoritmos

In [23]:
image = cv2.imread('../data/imgs/dsc07631.jpg')
h_kp, harris_image = detectHarrisKeypoints(image)
s_kp, sift_image = detectSIFTKeypoints(image)
st_kp, star_image = detectStarKeypoints(image)


### threshold para matches de detectores 2

In [24]:
matches_hs = computeDistacesKeypoints(h_kp, s_kp, threshold=2)
matches_hst = computeDistacesKeypoints(h_kp, st_kp, threshold=2)
matches_sst = computeDistacesKeypoints(s_kp, st_kp, threshold=2)

In [26]:
print('----- Resultados -----')
print(f'QTD Harris Keypoints:   {len(h_kp)}')
print(f'QTD SIFT Keypoints:   {len(s_kp)}')
print(f'QTD Star Keypoints:   {len(st_kp)}')
print('----- Matches -----')
print(f'Matches Keypoints Harris/SIFT:   {len(matches_hs)}')
print(f'Matches Keypoints Harris/Star:   {len(matches_hst)}')
print(f'Matches Keypoints SIFT/Star:   {len(matches_sst)}')

----- Resultados -----
QTD Harris Keypoints:   417
QTD SIFT Keypoints:   1518
QTD Star Keypoints:   291
----- Matches -----
Matches Keypoints Harris/SIFT:   80
Matches Keypoints Harris/Star:   14
Matches Keypoints SIFT/Star:   232


In [27]:
kp1_image = cv2.drawKeypoints(image, matches_hs, None, color=(0, 0, 255), flags=0)
kp2_image = cv2.drawKeypoints(image, matches_hst, None, color=(0, 255, 0), flags=0)
kp3_image = cv2.drawKeypoints(image, matches_sst, None, color=(255, 0, 0), flags=0)

In [29]:
cv2.imshow('Matches Harris vs SIFT Keypoints', kp1_image)
cv2.imshow('Matches Harris vs STAR Keypoints', kp2_image)
cv2.imshow('Matches SIFT vs STAR Keypoints', kp3_image)
cv2.waitKey()

-1

# Comparando Descritores

In [7]:
def calculate_slope(pt1, pt2):
    slope = (pt2[1] - pt1[1]) / (pt2[0] - pt1[0])
    return abs(slope)


def verify_slope(match, trainKeypoints, queryKeypoints, q1, q2):
    slope = calculate_slope(trainKeypoints[match.trainIdx].pt, queryKeypoints[match.queryIdx].pt) 
    if slope <= q2 and slope > q1:
        return True
    else:
        return False


def filter_matches_by_slope(matches, matchesMask, trainKeypoints, queryKeypoints, q1, q2):
    slope_list = [calculate_slope(trainKeypoints[matches[i][0].trainIdx].pt, queryKeypoints[matches[i][0].queryIdx].pt) for i, m in enumerate(matchesMask) if m[0] > 0]
    q1_, q2_ = np.percentile(slope_list, [q1 ,q2])
    matchesMask_ = [ [1,0] if m[0] > 0 and verify_slope(matches[i][0], trainKeypoints, queryKeypoints, q1_, q2_) else [0,0] for i, m in enumerate(matchesMask)]
    
    qtd_fp = len([m[0] for m, m_ in zip(matchesMask, matchesMask_) if m[0] != m_[0]])
    qtd_matches = len([m[0] for m in matchesMask_ if m[0] > 0 ])
    
    mismatch_train_keypoints = len(trainKeypoints) - qtd_matches
    mismatch_query_keypoints = len(queryKeypoints) - qtd_matches

    return matchesMask_, qtd_fp, qtd_matches, mismatch_train_keypoints, mismatch_query_keypoints


In [8]:
def matchDetectedKeypoints(train_img, query_img, ratio_test=0.7, filter_slope=(2,98), detector='harris', descriptor='orb', dect_kargs={}, des_kargs={}):
    query_img_bw = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY) 
    train_img_bw = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY)

    if descriptor == 'brief':
        des = cv2.xfeatures2d.BriefDescriptorExtractor_create(**des_kargs)
    else:
        des = cv2.SIFT.create(**des_kargs)
    
    if detector == 'star':
        func_var = detectStarKeypoints
    elif detector == 'harris':
        func_var = detectHarrisKeypoints
    else:
        func_var = detectSIFTKeypoints

    queryKeypoints, _ = func_var(query_img, **dect_kargs)
    trainKeypoints, _ = func_var(train_img, **dect_kargs)
    
    _,queryDescriptors = des.compute(query_img_bw, queryKeypoints, **des_kargs) 
    _,trainDescriptors = des.compute(train_img_bw, trainKeypoints, **des_kargs)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=100)   # or pass empty dictionary

    # flann = cv2.FlannBasedMatcher(index_params, search_params)
    # matches = flann.knnMatch(queryDescriptors, trainDescriptors, k=2)

    matcher = cv2.BFMatcher() 
    matches = matcher.knnMatch(queryDescriptors,trainDescriptors, k=2) 

    # Need to draw only good matches, so create a mask
    matchesMask = [[0,0] for i in range(len(matches))]

    for i,(m,n) in enumerate(matches):
        if m.distance < ratio_test*n.distance:
            matchesMask[i]=[1,0]

    m = filter_matches_by_slope(matches, matchesMask, trainKeypoints, queryKeypoints, filter_slope[0], filter_slope[1])
    matchesMask_, qtd_fp, qtd_matches, mismatch_train_keypoints, mismatch_query_keypoints = m 

    print('---- Resultado ---- ')
    print(f'Quantidade de matches: {qtd_matches}')
    print(f'Quantidade de falso positivos: {qtd_fp}')
    print(f'Quantidade de pontos sem match (train_image): {mismatch_train_keypoints}')
    print(f'Quantidade de pontos sem match (query_image): {mismatch_query_keypoints}')

    draw_params = dict(matchColor = (0,255,0),
                    singlePointColor = (255,0,0),
                    matchesMask = matchesMask_,
                    flags = cv2.DrawMatchesFlags_DEFAULT)
    final_img = cv2.drawMatchesKnn(query_img,queryKeypoints,train_img,trainKeypoints,matches,None,**draw_params)

    
    final_img = cv2.resize(final_img, (1280,960))

    return final_img 

    

In [9]:
query_img = cv2.imread('../data/imgs/dsc07632.jpg') 
train_img = cv2.imread('../data/imgs/dsc07631.jpg') 
query_img_ = cv2.resize(query_img, (640,480))
train_img_ = cv2.resize(train_img, (640,480))

In [10]:
params_sift = {
    'nfeatures': 100,
    'nOctaveLayers' : 3,
    'contrastThreshold' : 0.04,
    'edgeThreshold' : 20,
    'sigma' : 1.6
}

In [15]:
final_img = matchDetectedKeypoints(train_img_, query_img_, 0.7, filter_slope=(0,75), detector='harris', descriptor='sift', dect_kargs={}, des_kargs={})

---- Resultado ---- 
Quantidade de matches: 234
Quantidade de falso positivos: 79
Quantidade de pontos sem match (train_image): 183
Quantidade de pontos sem match (query_image): 176


In [16]:
# Show the final image 
cv2.imshow("Matches", final_img) 
cv2.waitKey()

-1

# Comparando Fluxo Óptico

## Lucas-Kanade

In [92]:
import numpy as np 
import cv2 

# train_img = cv2.imread('../data/imgs_teste/IMG_20231129_083404.jpg') 
# query_img = cv2.imread('../data/imgs_teste/IMG_20231129_083413.jpg')
query_img = cv2.imread('../data/imgs/dsc07632.jpg') 
train_img = cv2.imread('../data/imgs/dsc07631.jpg')  
train_img_ = cv2.resize(train_img, (480,640))
query_img_ = cv2.resize(query_img, (480,640))

# params for corner detection 
feature_params = dict( maxCorners = 100, 
                       qualityLevel = 0.2, 
                       minDistance = 7, 
                       blockSize = 7,
                       useHarrisDetector = False) 
  
# Parameters for lucas kanade optical flow 
lk_params = dict( winSize = (25, 25), 
                  maxLevel = 2, 
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 
                              10, 0.03)) 
  
# Create some random colors 
color = np.random.randint(0, 255, (100, 3)) 
  
# Take first frame and find corners in it 
old_frame = train_img_ 
old_gray = cv2.cvtColor(old_frame, 
                        cv2.COLOR_BGR2GRAY) 
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, 
                             **feature_params) 

# Create a mask image for drawing purposes 
mask = np.zeros_like(old_frame, 'uint8') 
  

frame = query_img_ 
frame_gray = cv2.cvtColor(frame, 
                            cv2.COLOR_BGR2GRAY) 

# calculate optical flow 
p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, 
                                        frame_gray, 
                                        p0, None, 
                                        **lk_params) 

# Select good points 
good_new = p1[st == 1] 
good_old = p0[st == 1] 

# draw the tracks 
for i, (new, old) in enumerate(zip(good_new,  
                                    good_old)):
    a, b = new.ravel() 
    c, d = old.ravel()
    
    a, b = int(a), int(b)
    c, d = int(c), int(d)

    mask = cv2.line(mask, (a, b), (c, d), 
                    color[i].tolist(), 2) 
        
    frame = cv2.circle(frame, (a, b), 5, 
                        color[i].tolist(), -1) 
        
img = cv2.add(frame, mask) 

cv2.imshow('frame', img)
cv2.waitKey()

# # Updating Previous frame and points  
# old_gray = frame_gray.copy() 
# p0 = good_new.reshape(-1, 1, 2) 


-1

## Horn-Schunk 

In [93]:
def draw_flow(img, flow, step=16):
    h, w = img.shape[:2]
    y, x = np.mgrid[step / 2:h:step, step / 2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T
    lines = np.vstack([x, y, x + fx, y + fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(vis, lines, 0, (0, 255, 0))
    for (x1, y1), (x2, y2) in lines:
        cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis

In [94]:
# Importing libraries
import cv2
import numpy as np

# train_img = cv2.imread('../data/imgs_teste/IMG_20231129_083404.jpg') 
# query_img = cv2.imread('../data/imgs_teste/IMG_20231129_083413.jpg')
query_img = cv2.imread('../data/imgs/dsc07632.jpg') 
train_img = cv2.imread('../data/imgs/dsc07631.jpg')  
train_img_ = cv2.resize(train_img, (480,640))
query_img_ = cv2.resize(query_img, (480,640))

# Convert to gray scale
prvs = cv2.cvtColor(train_img_, cv2.COLOR_BGR2GRAY)
# Create mask
hsv_mask = np.zeros_like(train_img_)
# Make image saturation to a maximum value
hsv_mask[..., 1] = 255
 
# Capture another frame and convert to gray scale
next = cv2.cvtColor(query_img_, cv2.COLOR_BGR2GRAY)

# Optical flow is now calculated
flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

mapped_img = draw_flow(next, flow, step=16)

cv2.imshow('frame2', mapped_img)
cv2.waitKey()


-1